In [ ]:
import os
import kagglehub

dataset_path = kagglehub.dataset_download("kondwani/eye-disease-dataset")
BASE_DIR = os.path.join(dataset_path, "Eye_diseases")
print("Dataset path:", BASE_DIR)
classes = sorted(os.listdir(BASE_DIR))
print("Classes:", classes)


Dataset path: /kaggle/input/eye-disease-dataset/Eye_diseases
Classes: ['Bulging_Eyes', 'Cataracts', 'Crossed_Eyes', 'Glaucoma', 'Uveitis']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE   = (224, 224)
BATCH_SIZE = 16
VAL_SPLIT  = 0.2

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=classes,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=classes,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 308 images belonging to 5 classes.
Found 75 images belonging to 5 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0

base = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(*IMG_SIZE, 3)
)
x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(classes), activation='softmax')(x)
model = Model(inputs=base.input, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,055,976 (15.47 MB)

 Trainable params: 4,013,953 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint(
    'best_efficientnet_eye.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
EPOCHS = 20

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop]
)
print("Training finished. Best model saved as best_efficientnet_eye.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2478 - loss: 1.6418
Epoch 1: val_accuracy improved from -inf to 0.08000, saving model to best_efficientnet_eye.h5


20/20 ━━━━━━━━━━━━━━━━━━━━ 122s 3s/step - accuracy: 0.2502 - loss: 1.6380 - val_accuracy: 0.0800 - val_loss: 1.6483
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.5359 - loss: 1.2839
Epoch 2: val_accuracy improved from 0.08000 to 0.13333, saving model to best_efficientnet_eye.h5


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 291ms/step - accuracy: 0.5371 - loss: 1.2823 - val_accuracy: 0.1333 - val_loss: 1.6926
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.5885 - loss: 1.0979
Epoch 3: val_accuracy did not improve from 0.13333
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 289ms/step - accuracy: 0.5891 - loss: 1.0968 - val_accuracy: 0.1200 - val_loss: 1.7315
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.6563 - loss: 0.9445
Epoch 4: val_accuracy did not improve from 0.13333
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.6583 - loss: 0.9433 - val_accuracy: 0.1200 - val_loss: 1.7749
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 0.7147 - loss: 0.8349
Epoch 5: val_accuracy did not improve from 0.13333
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 286ms/step - accuracy: 0.7148 - loss: 0.8338 - val_accuracy: 0.1200 - val_loss: 1.8328
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.7550 - loss: 0.7036
Epoch 6: val_accuracy did not impro

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

best_model = load_model('best_efficientnet_eye.h5', compile=False)
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
val_gen.reset()
y_true = val_gen.classes
y_pred_probs = best_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
class_names = [name for name, idx in sorted(train_gen.class_indices.items(), key=lambda x: x[1])]
print("\nClassification Report on Validation Set:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step

Classification Report on Validation Set:

              precision    recall  f1-score   support

Bulging_Eyes       0.00      0.00      0.00         6
   Cataracts       0.00      0.00      0.00         9
Crossed_Eyes       0.00      0.00      0.00        34
    Glaucoma       0.00      0.00      0.00        16
     Uveitis       0.13      1.00      0.24        10

    accuracy                           0.13        75
   macro avg       0.03      0.20      0.05        75
weighted avg       0.02      0.13      0.03        75



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

best_model = load_model('best_mobilenet_eye.h5', compile=False)
best_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
val_gen.reset()
y_true = val_gen.classes
y_pred_probs = best_model.predict(val_gen, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
class_names = [name for name, idx in sorted(train_gen.class_indices.items(), key=lambda x: x[1])]
print("\nClassification Report on Validation Set:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 618ms/step

Classification Report on Validation Set:

              precision    recall  f1-score   support

Bulging_Eyes       0.00      0.00      0.00         6
   Cataracts       0.88      0.78      0.82         9
Crossed_Eyes       0.72      0.85      0.78        34
    Glaucoma       0.61      0.69      0.65        16
     Uveitis       0.57      0.40      0.47        10

    accuracy                           0.68        75
   macro avg       0.56      0.54      0.54        75
weighted avg       0.64      0.68      0.65        75

